In [1]:
import csv

In [2]:
import matplotlib.pyplot as plt

In [3]:
def chop(x):
    return x.split("/")[-1].split(".")[0]

In [3]:
import numpy as np

In [4]:
def groundTruth(csvfile):
    f = open(csvfile)
    rows = [row for row in csv.reader(f)]
    rows = rows[1:]
    f.close()
    isClone = {}
    for a, b, clonep in rows:
        if clonep.strip() == "True":
            clonep = True
        elif clonep.strip() == "False":
            clonep = False
        else:
            raise Exception("%s is not a Boolean!" % clonep)
        isClone[(a.strip(), b.strip())] = clonep
    return isClone

In [5]:
isClone = groundTruth("experiment-results/same-file-pairs/ground-truth-antlr-js-java.csv")

In [6]:
def getData(csvfile, isClone, isDistance=False):
    f = open(csvfile)
    rows = [row for row in csv.reader(f)]
    rows = rows[1:]
    f.close()
    if isDistance:
        fScore = lambda x: 1 - float(x)
    else:
        fScore = float
    data = [(fScore(row[2].strip()), isClone[(row[0].strip(), row[1].strip())]) for row in rows]
    data.sort()
    clones = [int(r[1]) for r in data]
    scores = [r[0] for r in data]
    return scores, clones

In [46]:
scores, clones = getData("experiment-results/antlr-param-search/filtered/test_5_-2_commented.json.csv", isClone)

In [52]:
theirScores, theirClones = getData("experiment-results/antlr/mining.csv", isClone, True)

In [53]:
badScores, badClones = getData("experiment-results/antlr-param-search/filtered/test_0.5_-4_commented.json.csv", isClone)

In [10]:
plt.plot(scores, np.cumsum(clones), 'b-')

In [11]:
plt.plot(theirScores, np.cumsum(theirClones), 'r-')

In [12]:
plt.show()

In [135]:
definitelyClones = [i for i in isClone if isClone[i]]

In [136]:
len(definitelyClones)

24

In [29]:
np.sum(theirClones)

13

In [13]:
def findBin(x, nBins):
    return np.floor(x * 10)

In [74]:
def findRatios(scores, clones, nBins):
    bins = np.zeros(nBins)
    nonCloneBins = np.zeros(nBins)
    for s, c in zip(scores, clones):
        if c:
            bins[findBin(s, nBins)] += 1
        else:
            nonCloneBins[findBin(s, nBins)] += 1
    cbins = np.cumsum(bins[::-1])
    cnonCloneBins = np.cumsum(nonCloneBins[::-1])
    return np.nan_to_num(cbins / (cbins + cnonCloneBins))

In [75]:
theirs = findRatios(theirScores, theirClones, 10)

/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:8: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:6: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:11: RuntimeWarning: invalid value encountered in true_divide


In [76]:
ours = findRatios(scores, clones, 10)

/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:8: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:6: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:11: RuntimeWarning: invalid value encountered in true_divide


In [77]:
bads = findRatios(badScores, badClones, 10)

/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:8: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:6: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:11: RuntimeWarning: invalid value encountered in true_divide


In [90]:
bins = np.linspace(0.0, 0.9, 10)

In [91]:
plt.plot(bins, ours, 'ro-')

In [92]:
plt.plot(bins, theirs, 'bo-')

In [93]:
plt.plot(bins, bads, 'ko-')

In [94]:
plt.plot(bins, np.linspace(0, ours[-1], 10), 'g--')

In [95]:
plt.show()

In [67]:
for i in range(len(theirClones)):
    if theirScores[i] < 0.2 and theirClones[i]:
        print(i)
        

183


In [96]:
sp = [s for s, c in zip(scores, clones) if c]

In [97]:
sn = [s for s, c in zip(scores, clones) if not c]

In [144]:
plt.plot(np.zeros(len(sn)), sn, 'b+')

In [145]:
plt.plot(np.zeros(len(sp)), sp, 'rx')

In [146]:
tsp = [s for s, c in zip(theirScores, theirClones) if c]
tsn = [s for s, c in zip(theirScores, theirClones) if not c]
plt.plot(np.zeros(len(tsn))+1, tsn, 'b+')
plt.plot(np.zeros(len(tsp))+1, tsp, 'rx')

In [147]:
plt.xlim(-1,2)

(-1, 2)

In [148]:
plt.show()

In [150]:
colors=['red', 'blue'] * 2

In [177]:
bplot = plt.boxplot([sp, sn, tsp, tsn], notch=True, vert=True, patch_artist=True, whis='range', positions=[0, 0, 1, 1], labels=['Fett', 'Fett', 'CLCMiner', 'CLCMiner'])

In [178]:
for patch, color in zip(bplot['boxes'], colors):
    patch.set_facecolor(color)
for patch, color in zip(bplot['fliers'], colors):
    patch.set_color(color)
for patch, color in zip(bplot['whiskers'], ['red', 'red', 'blue', 'blue'] * 2):
    patch.set_color(color)

In [179]:
plt.show()

In [167]:
bplot

{'boxes': [<matplotlib.patches.PathPatch at 0x7f91480b6cc0>,
 'caps': [<matplotlib.lines.Line2D at 0x7f91480bce48>,
 'fliers': [<matplotlib.lines.Line2D at 0x7f91480c7e10>,
 'means': [],
 'medians': [<matplotlib.lines.Line2D at 0x7f91480c0eb8>,
 'whiskers': [<matplotlib.lines.Line2D at 0x7f91480bc358>,
  <matplotlib.lines.Line2D at 0x7f91480f4c50>]}